In [ ]:
!pip install unidecode
!pip install selenium
!pip install bs4
!pip install requests

# Scraping de los artículos de NewsRare por secciones (Selenium +BS4)

In [ ]:
import pandas as pd
from unidecode import unidecode
import json
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

En primer lugar tenemos que poder acceder a cada sección

In [ ]:
# Descargar la página
url = "https://newsrare.es/articulos"
response = requests.get(url)
# Parsear el HTML
soup = BeautifulSoup(response.text, "html.parser")
# Encontrar el bloque de secciones
section_container = soup.find("div", class_="td_block_categories_tags")
# Extraer los enlaces de las secciones
sections = section_container.find_all("a")

sections_urls=[]
nombre_seccion=[]
num_articulos=[]

for content in sections:

    section_url=content["href"]
    name=content.find("span",class_="td-ct-item-name").get_text()
    num=content.find("span",class_="td-ct-item-no").get_text()

    sections_urls.append(section_url)
    nombre_seccion.append(name)
    num_articulos.append(num)

df_secciones=pd.DataFrame({
    "Seccion":nombre_seccion,
    "NºArtículos":num_articulos,
    "URL":sections_urls
})


In [ ]:
df_secciones.head()

,Seccion,NºArtículos,URL
0,Actualidad,175,https://newsrare.es/seccion/actualidad/
1,Artículos,13,https://newsrare.es/seccion/articulos/
2,Barómetro,16,https://newsrare.es/seccion/barometro/
3,Colaboración Público-Privada,24,https://newsrare.es/seccion/entrevistas/colabo...
4,Cursos,14,https://newsrare.es/seccion/encuentros/cursos/


Acceder a los artículos de cada sección

# Obtención de artículos por seccion ( Es necesario usar Selenium para hacer scroll en la página y que cargen todos)

In [ ]:
def obtener_articulos_seccion(url_seccion,filepath):
    # Selenium configuration
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Correr en headless mode (sin UI)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # Path al WebDriver
    webdriver_path = "./chromedriver.exe"

    # Iniciar WebDriver
    service = Service(webdriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # URL de la página de "Actualidad"
    #url_actualidad = "https://newsrare.es/seccion/actualidad/"
    # Cargar la página
    driver.get(url_seccion)
    category=url_seccion.split("/")[-2]
    filename="noticias_"+category+".json"

    # Espera para asegurar que la página cargue completamente
    time.sleep(3)

    # Realizar scroll para cargar más artículos
    scroll_pause_time = 15  # Tiempo de espera entre desplazamientos
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Hacer scroll hasta el fondo de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        # Verificar si el scroll alcanzó el final de la página
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Una vez que la página ha cargado completamente, obtener el contenido HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Buscar los artículos en la categoría 'Actualidad'
    articles = soup.find_all("div", class_="td-module-meta-info")

    # Filtrar los artículos cuya categoría sea 'Actualidad'
    counter=0
    title_set=set()
    noticias_list=[]
    for article in articles:
        category_element = article.find("a", class_="td-post-category")
        category_element_clean = unidecode(category_element.get_text(strip=True).lower()).replace(" ", "-")
        if category_element and category in category_element_clean:
            a_element = article.find("a")
            h3_element = article.find("h3")
            url_article=h3_element.find("a")["href"]
            title=h3_element.get_text(strip=True)
            if title not in title_set: #chequeo de duplicados
                counter+=1
                title_set.add(title)  # Add title to the set
                #print(f"{counter}. {a_element.get_text(strip=True)} | {title} | URL: {url_article}")
                noticias_list.append({
                    "title": title,
                    "url": url_article
                })
    # Cerrar el navegador de Selenium después de terminar
    driver.quit()

    # Creamos json con todas las noticias
    with open(filepath+"/"+filename, "w") as file:
        json.dump(noticias_list, file)
    print("JSON Creado Correctamente: ",filename)

In [ ]:
df_secciones["URL"]
for url in df_secciones["URL"].values:
    print(url)
    filename=url.split("/")[-2]
    print (filename)

https://newsrare.es/seccion/actualidad/
actualidad
https://newsrare.es/seccion/articulos/
articulos
https://newsrare.es/seccion/barometro/
barometro
https://newsrare.es/seccion/entrevistas/colaboracion-publico-privada/
colaboracion-publico-privada
https://newsrare.es/seccion/encuentros/cursos/
cursos
https://newsrare.es/seccion/encuentros/dialogos/
dialogos


In [ ]:
for url in df_secciones["URL"]:
    obtener_articulos_seccion(url,"./articles")


JSON Creado Correctamente:  noticias_actualidad.json
JSON Creado Correctamente:  noticias_articulos.json
JSON Creado Correctamente:  noticias_barometro.json
JSON Creado Correctamente:  noticias_colaboracion-publico-privada.json
JSON Creado Correctamente:  noticias_cursos.json
JSON Creado Correctamente:  noticias_dialogos.json


## Obtención del texto para los artículos para todas las secciones

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

# Path de destino
articles_folder_path = "./articles"

def get_article_text(url):
    """Obtener el texto de un artículo de una URL"""
    try:
        # Realizar la solicitud HTTP para obtener el HTML de la página
        response = requests.get(url)

        if response.status_code == 200:
            # Parsear el HTML con BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Acceder a la info del artículo
            article_body = soup.find_all(class_="elementor-widget-container")
            article_text = ""
            for section in article_body:
                article_text += section.text + "\n"

            if article_text == "":  # El cuerpo del artículo puede estar en otra clase
                article_body = soup.find_all(class_="td-post-content")
                for section in article_body:
                    article_text += section.text + "\n"

            return article_text.strip()  # Eliminar espacios en blanco extra

        else:
            print(f"Error al acceder a la URL: {url}")
            return None

    except Exception as e:
        print(f"Error al obtener el artículo de {url}: {e}")
        return None

def update_json_files(folder_path):
    """Actualizar los archivos JSON con el texto del artículo"""
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                articles = json.load(f)

            # Actualizar cada entrada con el texto del artículo
            for article in articles:
                url = article.get("url")
                if url:
                    print(f"Procesando artículo: {article['title']}")
                    article_text = get_article_text(url)
                    if article_text:  # Evitar asignar valores vacíos o None
                        article["content"] = article_text

            # Guardar el archivo JSON actualizado (indentación corregida)
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(articles, f, ensure_ascii=False, indent=4)
                print(f"Archivo {filename} actualizado.")

# Llamar a la función para actualizar los archivos
update_json_files(articles_folder_path)


Procesando artículo: MEDICAMENTOS
Procesando artículo: ESPAÑA-OPINIÓN POSITIVA DE LA EMA PARA GARADACIMAB
Procesando artículo: ESPAÑA-BIOCRYST PRESENTA CUATRO COMUNICACIONES
Procesando artículo: Nueva ley de la ELA: atención 24 horas, protección a los cuidadores, evaluación de la discapacidad
Procesando artículo: Publicación del Informe sobre «Optimización de la participación de los pacientes en la evaluación de los medicamentos»
Procesando artículo: El valor de los medicamentos huérfanos trasciende la salud y calidad de vida del paciente
Procesando artículo: Los retos de la incorporación de innovación terapéutica en enfermedades raras en España
Procesando artículo: Desigualdad en Europa: España mejora en el acceso a medicamentos huérfanos, pero enfrenta retrasos significativos en su disponibilidad
Procesando artículo: PROCESOS REGULATORIOS PARA MEDICAMENTOS CONTRA ENFERMEDADES RARAS EN ESTADOS UNIDOS Y LA UNIÓN EUROPEA. FLEXIBILIDADES Y OPORTUNIDADES DE COLABORACIÓN
Procesando artícul

Cargar todos los artículos en un df común (Update all_articles_with_source.csv)

In [ ]:
import os
import json
import pandas as pd

# Folder containing your JSON files
json_folder = "/content/drive/MyDrive/AN_NO_STRUCT/TFM/articles"

# List to store extracted articles
articles_list = []

# Loop through each JSON file
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        with open(os.path.join(json_folder, filename), "r", encoding="utf-8") as f:
            data = json.load(f)
            # Extract articles and store their source file
            for article in data:
                articles_list.append({
                    "article_url": article.get("url", ""),  # Store original JSON filename
                    "title": article.get("title", ""),
                    "content": article.get("content", "")
                })

# Convert to Pandas DataFrame
df = pd.DataFrame(articles_list)

# Add combined text column for topic modeling
df["text"] = df["title"] + " " + df["content"]

# Drop empty texts
df = df[df["text"].str.strip() != ""]

# Display first few rows
df.head()

# Save DataFrame to CSV
df.to_csv("/content/drive/MyDrive/AN_NO_STRUCT/TFM/all_articles_with_source.csv", index=False, encoding="utf-8")

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/AN_NO_STRUCT/TFM/all_articles_with_source.csv")
df.head()